Adding new features here for a 'v2' dataset. 

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [42]:
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [3]:
tripsdf.iloc[0]

tripduration                               634
starttime                  2013-07-01 00:00:00
stoptime                   2013-07-01 00:10:34
start station id                           164
start station name             E 47 St & 2 Ave
start station latitude                 40.7532
start station longitude               -73.9703
end station id                             504
end station name               1 Ave & E 15 St
end station latitude                   40.7322
end station longitude                 -73.9817
bikeid                                   16950
usertype                              Customer
birth year                                  \N
gender                                       0
Name: 0, dtype: object

In [15]:
dt = datetime.datetime.strptime(
        '2013-07-01 00:00:00', '%Y-%m-%d %H:%M:%S'
        ).replace(tzinfo=pytz.timezone('US/Eastern'))
for i in range(1,9):
    print(dt.replace(day=i).strftime('%d, %w, %A'))

01, 1, Monday
02, 2, Tuesday
03, 3, Wednesday
04, 4, Thursday
05, 5, Friday
06, 6, Saturday
07, 0, Sunday
08, 1, Monday


In [16]:
tripsdf['start_dt'] = tripsdf['starttime'].map(
            lambda x: datetime.datetime.strptime(
                x, '%Y-%m-%d %H:%M:%S'
                ).replace(tzinfo=pytz.timezone('US/Eastern')))
weekdays = set(['Monday', 'Tuesday', 'Wednesday', 
                'Thursday', 'Friday'])
tripsdf['weekday'] = tripsdf['start_dt'].map(
            lambda x: x.strftime('%A') in weekdays)

In [20]:
tripsdf.weekday.value_counts()

True     634899
False    208517
Name: weekday, dtype: int64

In [33]:
hours = {'06': 0, # morning
 '07': 0,
 '08': 0,
 '09': 0,
 '10': 1, # lunch
 '11': 1,
 '12': 1,
 '13': 1,
 '14': 2,
 '15': 2,
 '16': 2,
 '17': 3,
 '18': 3,
 '19': 3,
 '20': 3,
 '21': 3,
 '22': 4,
 '23': 4,
 '00': 4,
 '01': 4,
 '02': 4,
 '03': 4,
 '04': 4,
 '05': 4
}
tripsdf['time_of_day'] = tripsdf['starttime'].map(lambda x:hours.get(x[11:13]))
#tripsdf['time_of_day'] = tripsdf['starttime'].iloc[0][11:13]

In [41]:
tripsdf['weekday'].map(lambda x:int(x)).value_counts()

1    634899
0    208517
Name: weekday, dtype: int64

In [38]:
tripsdf.usertype.value_counts().sum(), tripsdf.shape

(843416, (843416, 18))

In [43]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)


In [74]:
#np.allclose?
# np.hstack((X[:5,:3], np.resize(X[:5, 3], (5, 1)) )) #, X[:5])
X[:5, 3]
#X[:10,3]

array(['Customer', 'Customer', 'Subscriber', 'Subscriber', 'Subscriber'],
      dtype=object)

In [93]:
import fresh.preproc.v2 as pv2
import ipdb
reload(pv2)

<module 'fresh.preproc.v2' from '/opt/program/fresh/preproc/v2.py'>

In [ ]:
# these sparse matrices. run into problems like w/ hstack... 
# .. <843416x83 sparse matrix of type '<class 'numpy.float64'>'
# 	with 2530248 stored elements in Compressed Sparse Row format>
with ipdb.launch_ipdb_on_exception():
    X_transformed, y_enc, proc_dict = pv2.preprocess(
        X, y, neighborhoods, labeled=True)

ValueError('all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)')


> <__array_function__ internals>(6)concatenate()

ipdb> l

ipdb> u
> /opt/conda/lib/python3.7/site-packages/numpy/core/shape_base.py(343)hstack()
    342     if arrs and arrs[0].ndim == 1:
--> 343         return _nx.concatenate(arrs, 0)
    344     else:

ipdb> u
> <__array_function__ internals>(6)hstack()

ipdb> u
> /opt/program/fresh/preproc/v2.py(50)xform()
     49             ),
---> 50         X[:, 4:5]
     51         ))

ipdb> l
     45         proc_bundle['enc'].transform(X[:, :3]),
     46         np.resize(
     47             proc_bundle['usertype_le'].transform(X[:, 3]),
     48             (num_rows, 1)
     49             ),
---> 50         X[:, 4:5]
     51         ))
     52     if y:
     53         y_enc = proc_bundle['le'].transform(y)
     54     else:
     55         y_enc = None

ipdb> p proc_bundle['enc'].transform(X[:, :3]).shape, np.resize(proc_bundle['usertype_le']).transform(X[:, 3]), (num_rows, 1)).shape, X[:, 4:5].shape
*** SyntaxError: invalid syntax
ipdb>

In [86]:
(len(neighborhoods), X_transformed[0].toarray(),
 X_transformed[0].toarray().shape, X[0])
# 75 + 3 + 5 + 1 + 1 

(75,
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 1.]]),
 (1, 83),
 array(['Midtown East', 0, 4, 'Customer', 1], dtype=object))